In [1]:
import tensorflow as tf
print(tf.__version__)

c:\users\rohan\anaconda3\envs\flower\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\rohan\anaconda3\envs\flower\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\rohan\anaconda3\envs\flower\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\rohan\anaconda3\envs\flower\lib\site-packages\tensorflow\pyth

1.14.0


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
import keras
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet50
from keras.applications.resnet import decode_predictions
from keras.applications.resnet import preprocess_input
from keras.models import Model,Sequential
from keras.layers import Input,UpSampling2D,Flatten,BatchNormalization,Dense,Dropout,GlobalAveragePooling2D
from keras import optimizers
import tensorflow as tf
import numpy as np
import time
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from PIL import Image

Using TensorFlow backend.


In [4]:
IMAGE_SIZE = [150, 150]

In [5]:
folders = glob(r'E:\rohan\rethink\TRAIN\*')
print(len(folders))

6


In [6]:
# # Fixed for our dataset
# NUM_CLASSES = 6

# # Fixed for flowers color images
# CHANNELS = 3

# IMAGE_RESIZE = 150
# RESNET50_POOLING_AVERAGE = 'avg'
# DENSE_LAYER_ACTIVATION = 'softmax'
# OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# # Common accuracy metric for all outputs, but can use different metrics for different output
# LOSS_METRICS = ['accuracy']

# # EARLY_STOP_PATIENCE must be < NUM_EPOCHS
# NUM_EPOCHS = 10
# EARLY_STOP_PATIENCE = 3

# # These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# # Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
# STEPS_PER_EPOCH_TRAINING = 10
# STEPS_PER_EPOCH_VALIDATION = 10

# # These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# # NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
# BATCH_SIZE_TRAINING = 100
# BATCH_SIZE_VALIDATION = 100

In [7]:
resnet= ResNet50(include_top=False, weights='imagenet',input_shape=IMAGE_SIZE + [3])

In [8]:
from keras.models import Model,Sequential
model = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model.add(ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet'))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model.add(Dense(6, activation = 'softmax'))

# Say not to train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 12294     
Total params: 23,600,006
Trainable params: 12,294
Non-trainable params: 23,587,712
_________________________________________________________________


In [10]:
from keras import optimizers

# sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [11]:
# # Data Augmentation
# train_datagen = ImageDataGenerator(
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#      validation_split=0.1,
#     fill_mode='nearest')


# # Data Augmentation
# test_datagen = ImageDataGenerator(
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest')

In [17]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory(r'E:\rohan\rethink\TRAIN\augmented_data',
                                                 target_size = (150, 150),
                                                 batch_size = 64,
                                                subset = 'training',
                                                 class_mode = 'categorical')

Found 54000 images belonging to 6 classes.


In [18]:
val_set = train_datagen.flow_from_directory(r'E:\rohan\rethink\TRAIN\augmented_data',
                                            target_size = (150, 150),
                                            batch_size = 64,
                                           subset = 'validation',
                                            class_mode = 'categorical')

Found 6000 images belonging to 6 classes.


In [19]:
test_ds = test_datagen.flow_from_directory(
    directory=r'E:\rohan\rethink\.',
    batch_size = 7,
    seed = 42,
    shuffle = False,
    classes = ['test'],
    target_size = (150,150))

Found 3409 images belonging to 1 classes.


In [20]:
fit_history = model.fit_generator(
        train_set,
        steps_per_epoch = 100,
        epochs = 10,
        validation_data = val_set,
        validation_steps = 20)

Epoch 1/10
 14/100 [===>..........................] - ETA: 4:22 - loss: 2.0647 - accuracy: 0.5246

KeyboardInterrupt: 

In [ ]:
# model.save("res50.h5")
# print("Saved model to disk")

In [16]:
fit_history = model.fit_generator(
        train_set,
        steps_per_epoch = 1000,
        epochs = 20,
        validation_data = val_set,
        validation_steps = 500)

Epoch 1/20
1000/1000 [==============================] - 498s 498ms/step - loss: 0.3212 - accuracy: 0.8836 - val_loss: 0.6128 - val_accuracy: 0.8008
Epoch 2/20
1000/1000 [==============================] - 454s 454ms/step - loss: 0.3223 - accuracy: 0.8858 - val_loss: 1.3348 - val_accuracy: 0.7252
Epoch 3/20
1000/1000 [==============================] - 452s 452ms/step - loss: 0.3195 - accuracy: 0.8835 - val_loss: 0.7816 - val_accuracy: 0.7720
Epoch 4/20
1000/1000 [==============================] - 448s 448ms/step - loss: 0.3098 - accuracy: 0.8882 - val_loss: 0.5675 - val_accuracy: 0.7401
Epoch 5/20
1000/1000 [==============================] - 456s 456ms/step - loss: 0.3120 - accuracy: 0.8896 - val_loss: 1.0429 - val_accuracy: 0.7671
Epoch 6/20
1000/1000 [==============================] - 757s 757ms/step - loss: 0.3076 - accuracy: 0.8893 - val_loss: 0.6808 - val_accuracy: 0.7741
Epoch 7/20
1000/1000 [==============================] - 996s 996ms/step - loss: 0.3094 - accuracy: 0.8880 - val_

KeyboardInterrupt: 